In [2]:
# Assignment 1
# 60191667 유도진
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets import fetch_openml

In [3]:
mnist = fetch_openml('mnist_784')
x_data = mnist.data.astype('float32')
y_data = mnist.target.astype(int)

In [4]:
from sklearn.model_selection import train_test_split
import torch

device = "cuda"
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train.values), torch.tensor(X_test.values)
y_train, y_test = torch.tensor(y_train.values), torch.tensor(y_test.values)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([56000, 784])
torch.Size([14000, 784])
torch.Size([56000])
torch.Size([14000])


In [5]:
# LogisticRegression Model
class LogisticRegression(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)
  def forward(self, x):
    outputs = torch.sigmoid(self.linear(x))
    return outputs

In [11]:
# Hypter parameters
epochs = 10000
input_dim = 784
output_dim = 10
lr = 0.01
model = LogisticRegression(input_dim, output_dim)
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [12]:
loss_save_arr=[]
for i in range(epochs):
  # Train
  model.train()
  optimizer.zero_grad()
  output = model(X_train)
  loss = criterion(output, y_train.long())

  loss.backward()
  optimizer.step()
  loss_save_arr.append(loss.data)

  if (i%1000 == 0):
    print("=====")
    print("epoch ", i)
    print("loss ", loss.data)
    _, pred = torch.max(output.data, axis=1)
    print("train_accuracy : {:0.3f}".format(float((pred==y_train).sum())/y_train.size(0)))
    model.eval()
    with torch.no_grad():
      output = model(X_test)
      _, pred = torch.max(output.data, axis=1)
      print("test_accuracy : {:0.3f}".format(float((pred==y_test).sum())/y_test.size(0)))

=====
epoch  0
loss  tensor(2.4327, device='cuda:0')
train_accuracy : 0.094
test_accuracy : 0.135
=====
epoch  1000
loss  tensor(1.5618, device='cuda:0')
train_accuracy : 0.889
test_accuracy : 0.889
=====
epoch  2000
loss  tensor(1.5523, device='cuda:0')
train_accuracy : 0.898
test_accuracy : 0.895
=====
epoch  3000
loss  tensor(1.5480, device='cuda:0')
train_accuracy : 0.901
test_accuracy : 0.898
=====
epoch  4000
loss  tensor(1.5472, device='cuda:0')
train_accuracy : 0.903
test_accuracy : 0.899
=====
epoch  5000
loss  tensor(1.5437, device='cuda:0')
train_accuracy : 0.904
test_accuracy : 0.899
=====
epoch  6000
loss  tensor(1.5432, device='cuda:0')
train_accuracy : 0.905
test_accuracy : 0.900
=====
epoch  7000
loss  tensor(1.5418, device='cuda:0')
train_accuracy : 0.905
test_accuracy : 0.902
=====
epoch  8000
loss  tensor(1.5408, device='cuda:0')
train_accuracy : 0.906
test_accuracy : 0.902
=====
epoch  9000
loss  tensor(1.5397, device='cuda:0')
train_accuracy : 0.906
test_accuracy :

#### Q. Accuracy를 올리기 위해서 무엇을 추가할 수 있을 지 한줄로 작성
learing rate 값을 여러 가지로 변경하면서 모델을 학습시켜본다. epoch값이 1000일 때와 9000일 때의 정확도의 큰 차이가 없다. 1000 epoch 이전에 minimum을 찾고 그 값으로 수렴했기 때문이다. 따라서 lr을 바꿔가며 다른 minimum이 있는 지 확인하는 과정을 통해서 성능을 개선시켜볼 수 있을 것 같다.